# My Generator Benchmark (Full)

This is the bigger benchmark notebook.

I use it to show deeper current progress:
1. feature support on more configs
2. config impact with more variants
3. speed scaling with rows and workers

This one can take longer than the light notebook.


In [7]:
import copy
import sys
import time
from pathlib import Path

import pandas as pd
from IPython.display import display

ROOT = Path.cwd()
if not (ROOT / 'src').exists() and (ROOT.parent / 'src').exists():
    ROOT = ROOT.parent

SRC = ROOT / 'src'
if SRC.exists() and str(SRC) not in sys.path:
    sys.path.insert(0, str(SRC))

from itergen import ItergenSynthesizer, RunConfig, get_sample_config
from itergen.schema.config import build_column_specs

pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 220)

print(f'Project root: {ROOT}')


def summarize_column_kinds(config):
    specs = build_column_specs(config)
    by_kind = {'binary': [], 'categorical': [], 'continuous': []}
    for col_id, spec in specs.items():
        by_kind.setdefault(spec.get('kind', 'other'), []).append(col_id)
    return specs, by_kind


def run_generation_case(label, config, run_overrides=None):
    params = dict(
        n_rows=2500,
        seed=101,
        tolerance=0.03,
        max_attempts=4,
        log_level='quiet',
        save_output=False,
        proposal_scoring_mode='incremental',
    )
    if run_overrides:
        params.update(run_overrides)

    run_cfg = RunConfig(**params)

    t0 = time.perf_counter()
    result = ItergenSynthesizer(config, run_cfg).generate()
    elapsed = time.perf_counter() - t0

    specs, by_kind = summarize_column_kinds(config)
    frame = result.dataframe

    summary = {
        'label': label,
        'rows': int(frame.shape[0]),
        'cols': int(frame.shape[1]),
        'binary_cols': len(by_kind.get('binary', [])),
        'categorical_cols': len(by_kind.get('categorical', [])),
        'continuous_cols': len(by_kind.get('continuous', [])),
        'success': bool(result.success),
        'attempts': int(result.attempts),
        'objective': float(result.metrics.get('objective', float('nan'))),
        'max_error': float(result.metrics.get('max_error', float('nan'))),
        'confidence': float(result.quality_report.get('confidence', float('nan'))),
        'runtime_sec': float(elapsed),
        'rows_per_sec': float(frame.shape[0] / elapsed) if elapsed > 0 else float('nan'),
    }
    return summary, result, specs, by_kind


def safe_first(items):
    return items[0] if items else None


Project root: /Users/temurmamanazarov/Work/Swansea-Uni/Code/itergen


In [2]:
PROFILE_NAME = 'full'

FEATURE_CASES = [
    ('Binary', 'binary', 3000),
    ('Categorical', 'categorical', 3000),
    ('Continuous', 'continuous', 3000),
    ('Continuous Parent Bins', 'continuous_parent_bins', 3000),
    ('Mixed', 'mixed', 3500),
    ('Mixed Large', 'mixed_large', 4000),
]

CONFIG_BASE_SAMPLE = 'mixed'
CONFIG_VARIANTS = [
    ('baseline_incremental', {}),
    ('stricter_tolerance', {'tolerance': 0.015, 'max_attempts': 6}),
    ('very_strict', {'tolerance': 0.01, 'max_attempts': 8}),
    ('faster_looser', {'tolerance': 0.06, 'max_attempts': 3}),
    ('full_scoring_mode', {'proposal_scoring_mode': 'full'}),
    ('attempt_workers_2', {'attempt_workers': 2}),
    ('attempt_workers_4', {'attempt_workers': 4}),
    ('small_group_lock', {'small_group_mode': 'lock'}),
    ('torch_controller_requested', {'use_torch_controller': True, 'torch_required': False}),
]

PERF_ROW_LEVELS = [1200, 2400, 4800]
PERF_WORKER_LEVELS = [1, 2, 4]
PERF_REPEATS = 3
PERF_MAX_ATTEMPTS = 5
PERF_OPTIMIZE_OVERRIDES = {
    'max_iters': 140,
    'patience': 10,
    'batch_size': 512,
    'proposals_per_batch': 24,
}

print(f'Loaded profile: {PROFILE_NAME}')


Loaded profile: full


## Part 1 - Feature check


In [3]:
feature_rows = []
feature_artifacts = {}

for label, sample, n_rows in FEATURE_CASES:
    print(f'Running feature case: {label}')
    cfg = get_sample_config(sample)
    summary, result, specs, by_kind = run_generation_case(
        label=label,
        config=cfg,
        run_overrides={'n_rows': n_rows, 'seed': 202, 'max_attempts': 5},
    )
    summary['sample'] = sample
    feature_rows.append(summary)
    feature_artifacts[sample] = {'result': result, 'specs': specs, 'by_kind': by_kind}
    print(f'Done: {label} in {summary["runtime_sec"]:.2f}s')

feature_df = pd.DataFrame(feature_rows).sort_values('runtime_sec').reset_index(drop=True)
display(feature_df[['label', 'sample', 'rows', 'cols', 'binary_cols', 'categorical_cols', 'continuous_cols', 'success', 'attempts', 'objective', 'max_error', 'confidence', 'runtime_sec', 'rows_per_sec']].round(4))

checks = []
for sample, payload in feature_artifacts.items():
    df = payload['result'].dataframe
    specs = payload['specs']
    by_kind = payload['by_kind']

    for col in by_kind.get('binary', []):
        observed = set(pd.Series(df[col]).dropna().unique().tolist())
        checks.append({'sample': sample, 'column': col, 'check': 'binary in {0,1}', 'ok': observed.issubset({0, 1})})

    for col in by_kind.get('categorical', []):
        allowed = set(specs[col].get('labels') or [])
        observed = set(pd.Series(df[col]).dropna().astype(str).unique().tolist())
        checks.append({'sample': sample, 'column': col, 'check': 'categorical in labels', 'ok': observed.issubset(allowed)})

    for col in by_kind.get('continuous', []):
        targets = specs[col].get('targets') or {}
        min_v = targets.get('min')
        max_v = targets.get('max')
        if min_v is None and max_v is None:
            continue
        series = pd.to_numeric(df[col], errors='coerce')
        bad = pd.Series(False, index=series.index)
        if min_v is not None:
            bad = bad | (series < float(min_v))
        if max_v is not None:
            bad = bad | (series > float(max_v))
        checks.append({'sample': sample, 'column': col, 'check': 'continuous min/max bound', 'ok': float(bad.mean()) <= 0.02})

checks_df = pd.DataFrame(checks)
if not checks_df.empty:
    checks_summary = checks_df.groupby('sample', as_index=False).agg(total=('ok', 'count'), passed=('ok', 'sum'))
    checks_summary['pass_rate'] = checks_summary['passed'] / checks_summary['total']
    display(checks_summary.sort_values('pass_rate', ascending=False).round(4))


Running feature case: Binary
Done: Binary in 0.55s
Running feature case: Categorical
Done: Categorical in 0.53s
Running feature case: Continuous
Done: Continuous in 3.29s
Running feature case: Continuous Parent Bins
Done: Continuous Parent Bins in 5.80s
Running feature case: Mixed
Done: Mixed in 6.55s
Running feature case: Mixed Large
Done: Mixed Large in 11.00s


,label,sample,rows,cols,binary_cols,categorical_cols,continuous_cols,success,attempts,objective,max_error,confidence,runtime_sec,rows_per_sec
0,Categorical,categorical,3000,2,0,2,0,True,1,0.0000,0.0000,1.0000,0.5252,5712.3666
1,Binary,binary,3000,3,3,0,0,True,1,0.0000,0.0000,1.0000,0.5500,5454.9500
2,Continuous,continuous,3000,2,0,0,2,True,1,0.0181,0.0355,0.9822,3.2916,911.4085
3,Continuous Parent Bins,continuous_parent_bins,3000,3,1,0,2,True,1,0.0129,0.0364,0.9878,5.7983,517.3939
4,Mixed,mixed,3500,4,2,1,1,True,1,0.0092,0.0344,0.9910,6.5523,534.1610
5,Mixed Large,mixed_large,4000,16,5,7,4,False,5,0.0350,0.1160,0.9680,10.9997,363.6451


,sample,total,passed,pass_rate
0,binary,3,3,1.0
1,categorical,2,2,1.0
2,continuous,2,2,1.0
3,continuous_parent_bins,2,2,1.0
4,mixed,4,4,1.0
5,mixed_large,14,14,1.0


## Part 2 - Config impact check


In [4]:
base_config = get_sample_config(CONFIG_BASE_SAMPLE)
variant_rows = []
variant_results = {}

for name, overrides in CONFIG_VARIANTS:
    print(f'Running variant: {name}')
    summary, result, specs, by_kind = run_generation_case(
        label=name,
        config=copy.deepcopy(base_config),
        run_overrides={
            'n_rows': 2800,
            'seed': 404,
            'tolerance': 0.03,
            'max_attempts': 5,
            **overrides,
        },
    )
    summary['variant'] = name
    variant_rows.append(summary)
    variant_results[name] = {'result': result, 'specs': specs, 'by_kind': by_kind}
    print(f'Done variant: {name} | objective={summary["objective"]:.4f} | runtime={summary["runtime_sec"]:.2f}s')

variants_df = pd.DataFrame(variant_rows).sort_values('objective').reset_index(drop=True)
base_obj = float(variants_df.loc[variants_df['variant'] == 'baseline_incremental', 'objective'].iloc[0])
base_runtime = float(variants_df.loc[variants_df['variant'] == 'baseline_incremental', 'runtime_sec'].iloc[0])
variants_df['objective_delta_vs_base'] = variants_df['objective'] - base_obj
variants_df['runtime_delta_vs_base_sec'] = variants_df['runtime_sec'] - base_runtime
display(variants_df[['variant', 'success', 'attempts', 'objective', 'objective_delta_vs_base', 'max_error', 'confidence', 'runtime_sec', 'runtime_delta_vs_base_sec']].round(4))

baseline = variant_results['baseline_incremental']
binary_col = safe_first(baseline['by_kind'].get('binary', []))
categorical_col = safe_first(baseline['by_kind'].get('categorical', []))
continuous_col = safe_first(baseline['by_kind'].get('continuous', []))

rows = []
for name, payload in variant_results.items():
    df = payload['result'].dataframe
    row = {'variant': name}
    if binary_col and binary_col in df.columns:
        row[f'{binary_col}_rate'] = float(pd.to_numeric(df[binary_col], errors='coerce').mean())
    if categorical_col and categorical_col in df.columns:
        dist = df[categorical_col].value_counts(normalize=True)
        row[f'{categorical_col}_top'] = str(dist.index[0]) if not dist.empty else None
        row[f'{categorical_col}_top_share'] = float(dist.iloc[0]) if not dist.empty else float('nan')
    if continuous_col and continuous_col in df.columns:
        series = pd.to_numeric(df[continuous_col], errors='coerce')
        row[f'{continuous_col}_mean'] = float(series.mean())
        row[f'{continuous_col}_std'] = float(series.std(ddof=0))
    rows.append(row)

display(pd.DataFrame(rows).sort_values('variant').reset_index(drop=True).round(4))


Running variant: baseline_incremental
Done variant: baseline_incremental | objective=0.0085 | runtime=5.51s
Running variant: stricter_tolerance
Done variant: stricter_tolerance | objective=0.0045 | runtime=7.17s
Running variant: very_strict
Done variant: very_strict | objective=0.0032 | runtime=8.13s
Running variant: faster_looser
Done variant: faster_looser | objective=0.0185 | runtime=2.38s
Running variant: full_scoring_mode
Done variant: full_scoring_mode | objective=0.0085 | runtime=5.32s
Running variant: attempt_workers_2
Done variant: attempt_workers_2 | objective=0.0085 | runtime=9.79s
Running variant: attempt_workers_4
Done variant: attempt_workers_4 | objective=0.0085 | runtime=10.47s
Running variant: small_group_lock
Done variant: small_group_lock | objective=0.0091 | runtime=4.90s
Running variant: torch_controller_requested
Done variant: torch_controller_requested | objective=0.0085 | runtime=5.13s


,variant,success,attempts,objective,objective_delta_vs_base,max_error,confidence,runtime_sec,runtime_delta_vs_base_sec
0,very_strict,True,1,0.0032,-0.0052,0.0118,0.9968,8.1267,2.6173
1,stricter_tolerance,True,1,0.0045,-0.0040,0.0168,0.9956,7.1653,1.6559
2,baseline_incremental,True,1,0.0085,0.0000,0.0327,0.9917,5.5094,0.0000
3,full_scoring_mode,True,1,0.0085,0.0000,0.0327,0.9917,5.3200,-0.1893
4,attempt_workers_2,True,1,0.0085,0.0000,0.0327,0.9917,9.7858,4.2765
5,attempt_workers_4,True,1,0.0085,0.0000,0.0327,0.9917,10.4679,4.9586
6,torch_controller_requested,True,1,0.0085,0.0000,0.0327,0.9917,5.1320,-0.3774
7,small_group_lock,True,1,0.0091,0.0006,0.0352,0.9911,4.9036,-0.6058
8,faster_looser,True,1,0.0185,0.0100,0.0733,0.9819,2.3830,-3.1264


,variant,loyalty_rate,device_top,device_top_share,spend_mean,spend_std
0,attempt_workers_2,0.4143,mobile,0.4750,109.8277,23.3633
1,attempt_workers_4,0.4143,mobile,0.4750,109.8277,23.3633
2,baseline_incremental,0.4143,mobile,0.4750,109.8277,23.3633
3,faster_looser,0.4118,mobile,0.4782,109.6825,21.3370
4,full_scoring_mode,0.4143,mobile,0.4750,109.8277,23.3633
5,small_group_lock,0.4132,mobile,0.4768,109.9069,23.2387
6,stricter_tolerance,0.4186,mobile,0.4736,109.8941,24.1616
7,torch_controller_requested,0.4143,mobile,0.4750,109.8277,23.3633
8,very_strict,0.4100,mobile,0.4736,109.5719,24.4117


## Part 3 - Performance check


In [5]:
perf_config = get_sample_config('mixed')
perf_rows = []

for n_rows in PERF_ROW_LEVELS:
    for workers in PERF_WORKER_LEVELS:
        for rep in range(PERF_REPEATS):
            print(f'Perf run -> rows={n_rows}, workers={workers}, repeat={rep}')
            seed = 900 + rep
            summary, _result, _specs, _by_kind = run_generation_case(
                label=f'rows_{n_rows}_w_{workers}_r_{rep}',
                config=copy.deepcopy(perf_config),
                run_overrides={
                    'n_rows': n_rows,
                    'seed': seed,
                    'tolerance': 0.03,
                    'max_attempts': PERF_MAX_ATTEMPTS,
                    'attempt_workers': workers,
                    'proposal_scoring_mode': 'incremental',
                    'optimize_overrides': PERF_OPTIMIZE_OVERRIDES,
                },
            )
            perf_rows.append({
                'n_rows': n_rows,
                'workers': workers,
                'repeat': rep,
                'runtime_sec': summary['runtime_sec'],
                'rows_per_sec': summary['rows_per_sec'],
                'objective': summary['objective'],
                'max_error': summary['max_error'],
                'success': summary['success'],
                'attempts': summary['attempts'],
            })

perf_df = pd.DataFrame(perf_rows)
display(perf_df.head(18).round(4))

perf_agg = perf_df.groupby(['n_rows', 'workers'], as_index=False).agg(
    avg_runtime_sec=('runtime_sec', 'mean'),
    median_runtime_sec=('runtime_sec', 'median'),
    avg_rows_per_sec=('rows_per_sec', 'mean'),
    avg_objective=('objective', 'mean'),
    avg_max_error=('max_error', 'mean'),
    success_rate=('success', 'mean'),
    avg_attempts=('attempts', 'mean'),
)

base = perf_agg[perf_agg['workers'] == 1][['n_rows', 'avg_runtime_sec']].rename(columns={'avg_runtime_sec': 'runtime_w1'})
perf_agg = perf_agg.merge(base, on='n_rows', how='left')
perf_agg['speedup_vs_1'] = perf_agg['runtime_w1'] / perf_agg['avg_runtime_sec']
display(perf_agg.round(4))

speedup_table = perf_agg.pivot(index='n_rows', columns='workers', values='speedup_vs_1')
display(speedup_table.round(3))

try:
    import matplotlib.pyplot as plt

    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    for workers in sorted(perf_agg['workers'].unique()):
        sub = perf_agg[perf_agg['workers'] == workers]
        axes[0].plot(sub['n_rows'], sub['avg_runtime_sec'], marker='o', label=f'workers={workers}')
        axes[1].plot(sub['n_rows'], sub['avg_rows_per_sec'], marker='o', label=f'workers={workers}')
    axes[0].set_title('Runtime vs Rows (Full)')
    axes[0].set_xlabel('n_rows')
    axes[0].set_ylabel('seconds')
    axes[0].legend()
    axes[1].set_title('Throughput vs Rows (Full)')
    axes[1].set_xlabel('n_rows')
    axes[1].set_ylabel('rows/sec')
    axes[1].legend()
    plt.tight_layout()
    plt.show()
except Exception as exc:
    print(f'Plot skipped: {exc}')

print('Current progress: full benchmark finished')


Perf run -> rows=1200, workers=1, repeat=0
Perf run -> rows=1200, workers=1, repeat=1
Perf run -> rows=1200, workers=1, repeat=2
Perf run -> rows=1200, workers=2, repeat=0
Perf run -> rows=1200, workers=2, repeat=1
Perf run -> rows=1200, workers=2, repeat=2
Perf run -> rows=1200, workers=4, repeat=0
Perf run -> rows=1200, workers=4, repeat=1
Perf run -> rows=1200, workers=4, repeat=2
Perf run -> rows=2400, workers=1, repeat=0
Perf run -> rows=2400, workers=1, repeat=1
Perf run -> rows=2400, workers=1, repeat=2
Perf run -> rows=2400, workers=2, repeat=0
Perf run -> rows=2400, workers=2, repeat=1
Perf run -> rows=2400, workers=2, repeat=2
Perf run -> rows=2400, workers=4, repeat=0
Perf run -> rows=2400, workers=4, repeat=1
Perf run -> rows=2400, workers=4, repeat=2
Perf run -> rows=4800, workers=1, repeat=0
Perf run -> rows=4800, workers=1, repeat=1
Perf run -> rows=4800, workers=1, repeat=2
Perf run -> rows=4800, workers=2, repeat=0
Perf run -> rows=4800, workers=2, repeat=1
Perf run ->

,n_rows,workers,repeat,runtime_sec,rows_per_sec,objective,max_error,success,attempts
0,1200,1,0,1.8832,637.2188,0.0096,0.0359,True,1
1,1200,1,1,2.0227,593.2572,0.0098,0.0371,True,1
2,1200,1,2,1.7846,672.4058,0.0093,0.0342,True,1
3,1200,2,0,4.3906,273.3090,0.0096,0.0359,True,1
4,1200,2,1,4.0094,299.2988,0.0098,0.0371,True,1
5,1200,2,2,3.8013,315.6844,0.0093,0.0342,True,1
6,1200,4,0,3.7130,323.1928,0.0096,0.0359,True,1
7,1200,4,1,3.8262,313.6295,0.0098,0.0371,True,1
8,1200,4,2,3.6260,330.9419,0.0093,0.0342,True,1
9,2400,1,0,3.5185,682.1115,0.0095,0.0364,True,1


,n_rows,workers,avg_runtime_sec,median_runtime_sec,avg_rows_per_sec,avg_objective,avg_max_error,success_rate,avg_attempts,runtime_w1,speedup_vs_1
0,1200,1,1.8969,1.8832,634.2939,0.0096,0.0358,1.0,1.0,1.8969,1.0000
1,1200,2,4.0671,4.0094,296.0974,0.0096,0.0358,1.0,1.0,1.8969,0.4664
2,1200,4,3.7217,3.7130,322.5881,0.0096,0.0358,1.0,1.0,1.8969,0.5097
3,2400,1,3.4801,3.5185,691.2779,0.0097,0.0365,1.0,1.0,3.4801,1.0000
4,2400,2,4.9593,4.0371,535.5442,0.0097,0.0365,1.0,1.0,3.4801,0.7017
5,2400,4,7.2202,7.2002,332.5320,0.0097,0.0365,1.0,1.0,3.4801,0.4820
6,4800,1,7.3309,7.2564,654.9062,0.0097,0.0371,1.0,1.0,7.3309,1.0000
7,4800,2,10.7704,8.7227,474.9246,0.0097,0.0371,1.0,1.0,7.3309,0.6807
8,4800,4,12.4206,14.1461,418.4914,0.0097,0.0371,1.0,1.0,7.3309,0.5902


workers,1,2,4
n_rows,,,
1200,1.0,0.466,0.510
2400,1.0,0.702,0.482
4800,1.0,0.681,0.590


Plot skipped: No module named 'matplotlib'
Current progress: full benchmark finished


## Output explanation and result notes

How to read output values:
- `objective`: main quality score; lower is better
- `max_error`: largest error point; lower is better
- `confidence`: quick quality indicator; higher is better
- `runtime_sec`: wall time for the run
- `rows_per_sec`: throughput
- `speedup_vs_1`: gain vs one worker

Possible causes behind results:
- more features and conditions increase optimization work
- strict tolerance usually improves fit but can slow down runtime
- worker scaling depends on workload size and process overhead
- run-to-run differences can appear due to search path and randomness


In [6]:
print('=== Full run discussion ===')

if 'feature_df' in globals() and len(feature_df) > 0:
    fast = feature_df.sort_values('runtime_sec').iloc[0]
    slow = feature_df.sort_values('runtime_sec', ascending=False).iloc[0]
    best = feature_df.sort_values('objective').iloc[0]
    print(f'- Fastest feature case: {fast["label"]} ({fast["runtime_sec"]:.2f}s).')
    print(f'- Slowest feature case: {slow["label"]} ({slow["runtime_sec"]:.2f}s).')
    print(f'- Best feature quality: {best["label"]} (objective={best["objective"]:.4f}).')
    print('  Possible cause: large or highly dependent configs need more optimization passes.')

if 'variants_df' in globals() and len(variants_df) > 0:
    best_variant = variants_df.sort_values('objective').iloc[0]
    fast_variant = variants_df.sort_values('runtime_sec').iloc[0]
    print(f'- Best variant quality: {best_variant["variant"]} (objective={best_variant["objective"]:.4f}).')
    print(f'- Fastest variant runtime: {fast_variant["variant"]} ({fast_variant["runtime_sec"]:.2f}s).')

    base_rows = variants_df[variants_df['variant'] == 'baseline_incremental']
    if len(base_rows) > 0:
        base = base_rows.iloc[0]
        dq = best_variant['objective'] - base['objective']
        dt = fast_variant['runtime_sec'] - base['runtime_sec']
        print(f'- Delta vs baseline (quality best): {dq:.4f} objective points.')
        print(f'- Delta vs baseline (fastest runtime): {dt:.2f}s.')
        print('  Possible cause: stronger rules/settings improve fit but can cost time.')

if 'perf_agg' in globals() and len(perf_agg) > 0:
    best_speed = perf_agg.sort_values('speedup_vs_1', ascending=False).iloc[0]
    print(
        f'- Best worker scaling: rows={int(best_speed["n_rows"])}, workers={int(best_speed["workers"])}, '
        f'speedup={best_speed["speedup_vs_1"]:.2f}x'
    )
    if best_speed['speedup_vs_1'] > 1.5:
        print('  Cause idea: workload is large enough to amortize process overhead.')
    elif best_speed['speedup_vs_1'] > 1.1:
        print('  Cause idea: parallelism helps, but overhead still limits perfect scaling.')
    else:
        print('  Cause idea: overhead or contention may hide worker benefits here.')

print('Current progress note: full results discussed.')


=== Full run discussion ===
- Fastest feature case: Categorical (0.53s).
- Slowest feature case: Mixed Large (11.00s).
- Best feature quality: Categorical (objective=0.0000).
  Possible cause: large or highly dependent configs need more optimization passes.
- Best variant quality: very_strict (objective=0.0032).
- Fastest variant runtime: faster_looser (2.38s).
- Delta vs baseline (quality best): -0.0052 objective points.
- Delta vs baseline (fastest runtime): -3.13s.
  Possible cause: stronger rules/settings improve fit but can cost time.
- Best worker scaling: rows=1200, workers=1, speedup=1.00x
  Cause idea: overhead or contention may hide worker benefits here.
Current progress note: full results discussed.


## My final conclusion (fill this after run)

Use 4-5 short lines:
- Current progress: ...
- Best quality result: ...
- Best speed result: ...
- Main trade-off I saw: ...
- Possible cause of this behavior: ...
- Next thing I will improve: ...
